In [1]:
### Fixing import errors of the

import sys
import os

# This code navigates up one directory from the notebook's location ('examples/')
# to get the project's root directory.
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# We check if the path is already in the system path.
# If not, we add it to the beginning of the list.
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added project root to Python path: {project_root}")
else:
    print(f"Project root is already in Python path: {project_root}")

# Optional: You can print the first few paths to verify
print("\nVerifying sys.path:")
for i, path in enumerate(sys.path[:5]):
    print(f"{i}: {path}")

Added project root to Python path: /home/nick/projects/Llama-Index-GliREL-GraphRAG

Verifying sys.path:
0: /home/nick/projects/Llama-Index-GliREL-GraphRAG
1: /usr/lib/python312.zip
2: /usr/lib/python3.12
3: /usr/lib/python3.12/lib-dynload
4: 


In [2]:
import json
from llama_index.core import Document
from src.GlirelPathExtractor import GlirelPathExtractor 
from llama_index.core import SimpleDirectoryReader, PropertyGraphIndex,Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
import nest_asyncio


In [3]:
llm = Ollama(
    model= "gemma3:12b",
    request_timeout=120.0,
    context_window=8128,
    temperature=0.0
)

Settings.llm = llm
Settings.chunk_size=512
Settings.chunk_overlap=64

embed_model = OllamaEmbedding(
    model_name="snowflake-arctic-embed2:latest",
    ollama_additional_kwargs={"mirostat": 0},
)
Settings.embed_model = embed_model

In [4]:
with open('../.data/novel.json', 'r') as file:
    # Load the JSON data from the file into a Python object
    data = json.load(file)

In [5]:
print(data[0]["corpus_name"])

Novel-30752


In [6]:
documents = [Document(text=t["context"],id_=t["corpus_name"]) for t in data]

In [12]:
doc = documents[1]
doc.id_

'Novel-51410'

In [8]:
extractor = GlirelPathExtractor(device="cuda")


In [13]:
# for starting later
documents = documents[2:]
documents[0].id_

'Novel-58553'

In [14]:
nest_asyncio.apply()
for document in documents:
    index = PropertyGraphIndex.from_documents(
        documents=[document],
        kg_extractors=[extractor],
        use_async = False,   
    )
    novel_id = document.id_
    index.storage_context.persist(persist_dir=f"./.persistent_storage/gli/{novel_id}")


2025-07-17 16:31:00.921 | WARNING  | glirel.spacy_integration:__call__:97 - The input text must contain at least two entities; skipping...
2025-07-17 16:31:01.047 | WARNING  | glirel.spacy_integration:__call__:97 - The input text must contain at least two entities; skipping...
2025-07-17 16:31:01.158 | WARNING  | glirel.spacy_integration:__call__:97 - The input text must contain at least two entities; skipping...


NO ENTITIES FOUND FOR THIS NODE!!!


2025-07-17 16:31:01.602 | WARNING  | glirel.spacy_integration:__call__:97 - The input text must contain at least two entities; skipping...


NO ENTITIES FOUND FOR THIS NODE!!!


2025-07-17 16:31:02.310 | WARNING  | glirel.spacy_integration:__call__:97 - The input text must contain at least two entities; skipping...


NO ENTITIES FOUND FOR THIS NODE!!!


2025-07-17 16:31:04.729 | WARNING  | glirel.spacy_integration:__call__:97 - The input text must contain at least two entities; skipping...
2025-07-17 16:31:05.055 | WARNING  | glirel.spacy_integration:__call__:97 - The input text must contain at least two entities; skipping...
2025-07-17 16:31:05.207 | WARNING  | glirel.spacy_integration:__call__:97 - The input text must contain at least two entities; skipping...


NO ENTITIES FOUND FOR THIS NODE!!!


2025-07-17 16:31:16.112 | WARNING  | glirel.spacy_integration:__call__:97 - The input text must contain at least two entities; skipping...
2025-07-17 16:31:17.715 | WARNING  | glirel.spacy_integration:__call__:97 - The input text must contain at least two entities; skipping...


NO ENTITIES FOUND FOR THIS NODE!!!


Response(response='Mont St. Michel is located in France.', source_nodes=[NodeWithScore(node=TextNode(id_='cbd321a1-9abd-4dfc-b7fe-7f3bffb75bb5', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='Novel-8559', node_type='4', metadata={}, hash='765da7cb58d5e45fa30d733fd531923ecc7c7216a4d70339a07dff13dd7ec090'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='76e1a5fb-7ab5-4b56-9560-14465d1b3a2c', node_type='1', metadata={}, hash='2f9711693831b24b525013258abd380fb379fd5c8d6c2f86132c06f90dbe3347'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='d660b25f-79da-4dea-8d97-2a7ff0a3b3e8', node_type='1', metadata={}, hash='35a91092ad4c633e0bb6602003e24b7633bca5c4a1a181b03e845c33ba705e37')}, metadata_template='{key}: {value}', metadata_separator='\n', text='Here are some facts extracted from the provided text:\n\nLoudon -> visited -> France\n\nThese dimensions have b